In [ ]:
from Layer.CSP_MB_Layers_test_1 import *
from DataGenerator import load_data
from models.model_1 import *
from models.config_1 import CustomizeSmall, CustomizeLarge, MobileNetLarge, MobileNetSamll, EfficientNetB0
from Flops import get_flops

In [2]:
# 模型參數設定
# MobileNetV3Large 、 MobileNetV3Small 、 EfficientNetB0 、 CustomizeLarge 、 CustomizeSmall
backbone = 'CustomizeLarge'
# None SE CBAM CA
Attention_block_1 = None
Attention_block_2 = 'SE'
# Adam RMSprop CLR
LR_mode = 'Adam'
unfrozen = []

# 圖片大小
input_shape = None
# 類別數
num_classes = 0

# 訓練參數
BATCH_SIZE = 512
epoch_1 = 150
epoch_2 = 500
Dropout_rate = 0.5

# Load Dataset

In [3]:
# 100 Bird Species  or  325 Bird Species  or  cifar100 or cifar10
Dataset = "cifar10"

if Dataset == "325 Bird Species":
    dir = "./Dataset/325 Bird Species/"
    input_shape = (224,224,3)
    num_classes = 325
    trainset, valset, testset = load_data(Dataset, dir, input_shape, BATCH_SIZE)
    
elif Dataset == "100 Bird Species":
    dir = "./Dataset/100 Bird Species/"
    input_shape = (224,224,3)
    num_classes = 100
    trainset, valset, testset = load_data(Dataset, dir, input_shape, BATCH_SIZE)
    
elif Dataset == "cifar100":
    input_shape = (32,32,3)
    num_classes = 100
    datagen_train, datagen_val, x_train, y_train, x_val, y_val, x_test, y_test = load_data(Dataset, '', input_shape, num_classes)
    
elif Dataset == "cifar10":
    input_shape = (32,32,3)
    num_classes = 10
    datagen_train, datagen_val, x_train, y_train, x_val, y_val, x_test, y_test= load_data(Dataset, '', input_shape, num_classes)

(10000, 1)
Number of training samples:  40000
Number of validation samples:  10000


In [4]:
if  backbone == "CustomizeLarge":
    Specification, unfrozen = CustomizeLarge(Dataset)
elif backbone == "CustomizeSmall":
    Specification, unfrozen = CustomizeSmall(Dataset)
elif backbone == 'MobileNetV3Large':
    Specification, unfrozen = MobileNetLarge(Dataset)
elif backbone == 'MobileNetV3Small':
    Specification, unfrozen = MobileNetSamll(Dataset)
elif backbone == 'EfficientNetB0':
    Specification, unfrozen = EfficientNetB0(Dataset)

base_model = build_base_model(input_shape, Attention_block_1, Attention_block_2, Specification)
# base_model.summary()

# Build Model

In [5]:
predtions = predictions_head(base_model, num_classes, Dropout_rate)
model = build_model(base_model, predtions)
model.summary()
print("The FLOPs is:{}".format(get_flops(model)) ,flush=True)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['Conv1[0][0]']                  
 alization)                                                                                       
                                                                                                  
 Conv1hardswish (HardSwish)     (None, 32, 32, 16)   0           ['batch_normalization[0][0]

# Compile

In [6]:
INIT_LR = 0.00025
MAX_LR = 0.001
if Dataset == "325 Bird Species" or Dataset == "100 Bird Species":
    steps_per_epoch = trainset.samples // BATCH_SIZE
elif Dataset == "cifar10" or Dataset == "cifar100":
    steps_per_epoch = len(x_train) // BATCH_SIZE

In [7]:
from tensorflow.keras.losses import CategoricalCrossentropy

optimizer = optimizer_set(LR_mode=LR_mode,
                          INIT_LR=INIT_LR,
                          MAX_LR=MAX_LR,
                          steps_per_epoch=steps_per_epoch)

loss = CategoricalCrossentropy(label_smoothing=0.1)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Callback

In [8]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard,EarlyStopping,ReduceLROnPlateau

checkpoint_path = f'weights_V2/{backbone}/{Attention_block_2}_{LR_mode}_{Dataset}/'+ "cp-{epoch:04d}.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    verbose=1, 
    save_weights_only=True,
    save_best_only=True, 
    mode='max',
    save_freq='epoch'
    )

tensorboard = TensorBoard(log_dir=f"logs/{backbone}/{Attention_block_2}_{LR_mode}_{Dataset}/",
                          histogram_freq=0,
                          write_graph=True,
                          write_images=False,
                          update_freq="epoch",
                          profile_batch=2,
                          embeddings_freq=0,
                          embeddings_metadata=None,)

callbacks = [tensorboard, checkpoint]

# Training Model

- 如果選擇的是自定義模型，則需要從頭開始訓練整個模型
- 若選擇其他模型，則直接轉至Fine-tuning

## Training-1

In [9]:
if Dataset == "325 Bird Species" or Dataset == "100 Bird Species":
    history_ft = model.fit(trainset,
                        epochs=epoch_1,
                        validation_data=valset,
                        callbacks=callbacks,
                        # verbose=0
                        )
elif Dataset == "cifar10" or Dataset == "cifar100":
    history_ft = model.fit(datagen_train.flow(x_train, y_train, batch_size=BATCH_SIZE),
                            epochs=epoch_1,
                            validation_data=datagen_val.flow(x_val, y_val, batch_size=BATCH_SIZE),
                            callbacks=callbacks,
                            # verbose=0
                            )

Epoch 1/150
79/79 [==============================] - ETA: 0s - loss: 2.2520 - accuracy: 0.2479
Epoch 1: val_accuracy improved from -inf to 0.10000, saving model to weights_V2/CustomizeLarge/SE_Adam_cifar10\cp-0001.ckpt
79/79 [==============================] - 25s 193ms/step - loss: 2.2520 - accuracy: 0.2479 - val_loss: 2.4022 - val_accuracy: 0.1000
Epoch 2/150
79/79 [==============================] - ETA: 0s - loss: 1.9608 - accuracy: 0.3589
Epoch 2: val_accuracy did not improve from 0.10000
79/79 [==============================] - 12s 154ms/step - loss: 1.9608 - accuracy: 0.3589 - val_loss: 2.4109 - val_accuracy: 0.1000
Epoch 3/150
79/79 [==============================] - ETA: 0s - loss: 1.8030 - accuracy: 0.4320
Epoch 3: val_accuracy did not improve from 0.10000
79/79 [==============================] - 13s 167ms/step - loss: 1.8030 - accuracy: 0.4320 - val_loss: 2.4211 - val_accuracy: 0.1000
Epoch 4/150
79/79 [==============================] - ETA: 0s - loss: 1.6912 - accuracy: 0.486

# Fine tuned model

## 設定unfrozen的Layer

In [10]:
# setup_to_transfer_learning(base_model, unfrozen=unfrozen)
# base_model.summary()

# Training-2

In [11]:
# if Dataset == "325 Bird Species" or Dataset == "100 Bird Species":
#     history_ft = model.fit(trainset,
#                             epochs=epoch_1 + epoch_2,
#                             initial_epoch=epoch_1,
#                             validation_data=valset,
#                             callbacks=callbacks)
# elif Dataset == "cifar10" or Dataset == "cifar100":
#     history_ft = model.fit(datagen_train.flow(x_train, y_train, batch_size=BATCH_SIZE),
#                             epochs=epoch_1 + epoch_2,
#                             initial_epoch=epoch_1,
#                             validation_data=datagen_val.flow(x_val, y_val, batch_size=BATCH_SIZE),
#                             callbacks=callbacks,
#                             )

# SAVE Model

In [12]:
import os
path = f'./weights_V2/{backbone}/{Attention_block_2}_{LR_mode}_{Dataset}'
model_num = ''
print('Fine模型列表：')
for a in os.listdir(path):
    print(a)
    model_num = a
print('最新的model為：',model_num)
model_num = a[:-6]

Fine模型列表：
checkpoint
cp-0001.ckpt.data-00000-of-00001
cp-0001.ckpt.index
cp-0006.ckpt.data-00000-of-00001
cp-0006.ckpt.index
cp-0008.ckpt.data-00000-of-00001
cp-0008.ckpt.index
cp-0009.ckpt.data-00000-of-00001
cp-0009.ckpt.index
cp-0010.ckpt.data-00000-of-00001
cp-0010.ckpt.index
cp-0012.ckpt.data-00000-of-00001
cp-0012.ckpt.index
cp-0014.ckpt.data-00000-of-00001
cp-0014.ckpt.index
cp-0016.ckpt.data-00000-of-00001
cp-0016.ckpt.index
cp-0021.ckpt.data-00000-of-00001
cp-0021.ckpt.index
cp-0022.ckpt.data-00000-of-00001
cp-0022.ckpt.index
cp-0023.ckpt.data-00000-of-00001
cp-0023.ckpt.index
cp-0025.ckpt.data-00000-of-00001
cp-0025.ckpt.index
cp-0032.ckpt.data-00000-of-00001
cp-0032.ckpt.index
cp-0033.ckpt.data-00000-of-00001
cp-0033.ckpt.index
cp-0034.ckpt.data-00000-of-00001
cp-0034.ckpt.index
cp-0035.ckpt.data-00000-of-00001
cp-0035.ckpt.index
cp-0036.ckpt.data-00000-of-00001
cp-0036.ckpt.index
cp-0039.ckpt.data-00000-of-00001
cp-0039.ckpt.index
cp-0047.ckpt.data-00000-of-00001
cp-0047.ck

In [13]:
model.load_weights(f'./weights_V2/{backbone}/{Attention_block_2}_{LR_mode}_{Dataset}/{model_num}')

In [14]:
model.save(f'./weights_V2/{backbone}/{Attention_block_2}_{LR_mode}_{Dataset}/best_model')

INFO:tensorflow:Assets written to: ./weights_V2/CustomizeLarge/SE_Adam_cifar10/best_model\assets


INFO:tensorflow:Assets written to: ./weights_V2/CustomizeLarge/SE_Adam_cifar10/best_model\assets


# TFLite

In [15]:
# path to the SavedModel directory
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_dir = f'./weights_V2/{backbone}/{Attention_block_2}_{LR_mode}_{Dataset}/tflite/'
if not os.path.isdir(tflite_dir):
    os.makedirs(tflite_dir)

# Save the model.
with open(tflite_dir + "model.tflite", 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpdauajrry\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpdauajrry\assets


In [16]:
# import time
# stepNum = len(y_test)
# start = time.time()

# #利用模型進行預測，記得輸入的test_generator的shuffle內建要是False
# results = model.evaluate(x_test/255, y_test, batch_size=1)

# end = time.time()
# # Time elapsed
# seconds = end - start
# FPS = stepNum/seconds
# print("FPS:",FPS)